In [25]:
import numpy as np
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [26]:
scores_df = pd.read_csv('C:/Users/Gabriel/Documents/MIE368/Data/season_data.csv')
playoff_scores_df = pd.read_csv('C:/Users/Gabriel/Documents/MIE368/Data/playoff_data.csv')

In [27]:
scores_df.head()

,Date,Visitor,G_Vis,Home,G_Home,OT Ind,Season
0,2000-10-04,Colorado Avalanche,2.0,Dallas Stars,2.0,1,2001
1,2000-10-05,Ottawa Senators,4.0,Boston Bruins,4.0,1,2001
2,2000-10-05,Chicago Blackhawks,2.0,Buffalo Sabres,4.0,0,2001
3,2000-10-05,Detroit Red Wings,4.0,Calgary Flames,3.0,0,2001
4,2000-10-05,Vancouver Canucks,3.0,Philadelphia Flyers,6.0,0,2001


## Game Result Elo Adjustment

In [28]:
first_yr = scores_df[scores_df['Season'] == 2001]
Teams = first_yr['Home'].unique().tolist()

team_changes = {
    'Atlanta Thrashers': 'Winnipeg Jets',
    'Phoenix Coyotes': 'Arizona Coyotes',
    'Mighty Ducks of Anaheim': 'Anaheim Ducks'
    # Add more mappings as needed
}

K = 6

In [85]:
def calculate_prob_winning(home_team, away_team, elo_ratings):
    
    Elo_diff_home = elo_ratings[home_team]-elo_ratings[away_team] + 26  #add extra for home-ice advantage   
    #Elo_diff_away = elo_ratings[away_team] - elo_ratings[home_team]
    prob_win_home = 1/(10**(-1*Elo_diff_home/400)+1)
    #prob_win_away = 1/(10**(-1*Elo_diff_away/400)+1)
    prob_win_away = 1 - prob_win_home
    
    return prob_win_home, prob_win_away, Elo_diff_home

In [86]:
def margin_of_victory(home_goals, away_goals):
    
    mov = abs(home_goals - away_goals)
    mov_mult = 0.6686*np.log(mov)+0.8048
    
    return mov_mult

In [94]:
# Function to update ELO ratings after a game -- Could add input for elo_ratings dictionary 
def update_game_elo(home_team, away_team, home_goals, away_goals, season, df, idx, elo_ratings):
    
    # Map old team names to new ones
    home_team = team_changes.get(home_team, home_team)
    away_team = team_changes.get(away_team, away_team)
    
    #print('Home team is ' + str(home_team))
    #print('Away team is ' + str(away_team))

    if home_team not in elo_ratings:
        # New teams added past a certain year start with a different ELO rating
        elo_ratings[home_team] = 1490 if season >= 2005 else 1380
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1490 if season >= 2005 else 1380

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, elo_ratings)
    
    # Update DataFrame values
    df.at[idx, 'EloDiffHome'] = Elo_diff_home
    df.at[idx, 'EloDiffAway'] = -1 * Elo_diff_home
    df.at[idx, 'WinProbHome'] = home_prob
    df.at[idx, 'WinProbVis'] = away_prob
    
     # Update Pregame Favorite Multiplier
    if home_goals > away_goals:
        home_win = 1
        #away_win = 0
        winner_elo_diff = Elo_diff_home
    elif home_goals < away_goals:
        home_win = 0
        #away_win = 1
        winner_elo_diff = -1*Elo_diff_home
    else:
        return df, elo_ratings
    
    pre_g_fav_h = home_win - home_prob  
    #pre_g_fav_a = away_win - away_prob  
    
    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)
    
     # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(home_goals, away_goals)
    #print('home goals:' + str(home_goals))
    #print('away goals:' + str(away_goals))
    #print('mult is: ' + str(mov_multiplier))
    
     # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h 
    #elo_shift_a = K * mov_multiplier * pre_g_fav_a
    #print('elo shift is: ' + str(elo_shift_h))
    
    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h

    return df, elo_ratings

## End of Season Elo Adjustments

In [88]:
def elo_adjuster(df):
    
    # Initialize variables to store the final Elo ratings for the season
    final_elo_ratings = {}
    elo_ratings = {}
    season_weight = 0.7
    average_weight = 0.3
    current_season = None

    # Iterate over seasons and games
    for idx, row in df.iterrows():
        if current_season is None:
            current_season = row['Season']
    
        # Check if the season in the current row is different from the current season
        if row['Season'] != current_season:
            # Save the previous season's Elo ratings and calculate the starting Elo for the new season
            final_elo_ratings[current_season] = dict(elo_ratings)
            elo_ratings = {}  # Reset Elo ratings for the new season

            for team in final_elo_ratings[current_season]:
                prev_season_elo = final_elo_ratings[current_season][team]
                league_avg_elo = sum(final_elo_ratings[current_season].values()) / len(final_elo_ratings[current_season])
                starting_elo = (season_weight * prev_season_elo) + (average_weight * league_avg_elo)
                elo_ratings[team] = round(starting_elo, 2)

            current_season = row['Season']
            
        # Update Elo ratings for the game
        df, elo_ratings = update_game_elo(row['Home'], row['Visitor'], row['G_Home'], row['G_Vis'], row['Season'], df, idx, elo_ratings)

    # Calculate the final Elo ratings for the last season
    final_elo_ratings[current_season] = dict(elo_ratings)

    return df, elo_ratings, final_elo_ratings

In [92]:
seasons, elo, final_season_elos = elo_adjuster(scores_df)

In [93]:
elo

{'Dallas Stars': 1438.449568876547,
 'Colorado Avalanche': 1467.0041326779394,
 'Boston Bruins': 1530.397215501669,
 'Ottawa Senators': 1365.1050381874425,
 'Buffalo Sabres': 1373.1864106169082,
 'Chicago Blackhawks': 1280.4201757826659,
 'Calgary Flames': 1402.1947282646518,
 'Detroit Red Wings': 1329.8577019700033,
 'Philadelphia Flyers': 1307.0972745553063,
 'Vancouver Canucks': 1370.2816315577088,
 'Arizona Coyotes': 1289.5863386862557,
 'St. Louis Blues': 1371.011391435606,
 'Edmonton Oilers': 1468.3836695037394,
 'Florida Panthers': 1425.486802121703,
 'Anaheim Ducks': 1255.6070111408524,
 'Minnesota Wild': 1431.4151312241875,
 'New Jersey Devils': 1419.86898941051,
 'Montreal Canadiens': 1294.8165146511856,
 'Pittsburgh Penguins': 1392.0652070159683,
 'Nashville Predators': 1392.4566649394321,
 'San Jose Sharks': 1272.6965553339462,
 'Tampa Bay Lightning': 1427.353379517724,
 'New York Islanders': 1399.2857789281184,
 'Washington Capitals': 1362.2730179203713,
 'Los Angeles King

In [137]:
def forecast_game_elo(home_team, away_team, season, df, idx, elo_ratings, simulation_wins):

    # Map old team names to new ones
    home_team = team_changes.get(home_team, home_team)
    away_team = team_changes.get(away_team, away_team)

    if home_team not in elo_ratings:
        elo_ratings[home_team] = 1490 if season >= 2005 else 1380
    if away_team not in elo_ratings:
        elo_ratings[away_team] = 1490 if season >= 2005 else 1380

     # Calculate the probability of winning the game for each team
    home_prob, away_prob, Elo_diff_home = calculate_prob_winning(home_team, away_team, elo_ratings)

    # Update DataFrame values
    df.at[idx, 'EloDiffHome'] = Elo_diff_home
    df.at[idx, 'EloDiffAway'] = -1 * Elo_diff_home
    df.at[idx, 'WinProbHome'] = home_prob
    df.at[idx, 'WinProbVis'] = away_prob
    
    home_win = np.random.choice([0, 1], p=[away_prob, home_prob])

    # Calculate the probability of going into overtime
    OT_prob = 1/(1+np.exp(-1*(0.092596 - 0.001466*abs(Elo_diff_home))))

    # Calculate base scores for the home and away team
    score_home = 2.845905 + 0.003584*Elo_diff_home
    score_away = 2.845905 + 0.003584*(-1)*Elo_diff_home

    conditions_satisfied = 0 

    while conditions_satisfied == 0:
        
        # poisson distribution to shift scores
        poisson_score_home = np.random.poisson(max(0, score_home))
        poisson_score_away = np.random.poisson(max(0, score_home))

        # Ensure non-negative scores by replacing any negative values with zero
        poisson_score_home = np.maximum(0, poisson_score_home)
        poisson_score_away = np.maximum(0, poisson_score_away)

        home_goal_diff = poisson_score_home - poisson_score_away
        
        # does it satsify conditions?
        if OT_prob >= 0.5:
            if home_win == 1:
                if (abs(home_goal_diff) == 1) and (home_goal_diff > 0):
                    conditions_satisfied = 1
            if home_win == 0:
                if (abs(home_goal_diff) == 1) and (home_goal_diff < 0): 
                    conditions_satisfied = 1
        
        if OT_prob < 0.5:
            if home_win == 1:
                if home_goal_diff > 0:
                    conditions_satisfied = 1
            if home_win == 0:
                if home_goal_diff < 0: 
                    conditions_satisfied = 1

    #------------------------------------------------------------------------#
    if home_win == 1:
        winner_elo_diff = Elo_diff_home
        simulation_wins[home_team] += 1
        
    if home_win == 0:
        winner_elo_diff = -1*Elo_diff_home
        simulation_wins[away_team] += 1

    pre_g_fav_h = home_win - home_prob  
    auto_corr = 2.05/(winner_elo_diff*0.001 + 2.05)
    
    # Adjust ELO shift for margin of victory
    mov_multiplier = auto_corr * margin_of_victory(poisson_score_home, poisson_score_away)
 
    # Calculate ELO shift based on game result
    elo_shift_h = K * mov_multiplier * pre_g_fav_h 
  
    elo_ratings[home_team] += elo_shift_h
    elo_ratings[away_team] -= elo_shift_h

    if OT_prob < 0.5:
        df.at[idx, 'OT Ind'] = 0
    else: 
        df.at[idx, 'OT Ind'] = 1

    df.at[idx, 'G_Home'] = poisson_score_home
    df.at[idx, 'G_Vis'] = poisson_score_away
    #--------------------------------------------------------------------------#

    return df, elo_ratings, simulation_wins

In [138]:
# Iterate over seasons and games
def season_forecaster(df, starting_elo):
    
    simulation_wins = {team: 0 for team in df['Home'].unique()}

    for idx, row in df.iterrows():
        df, starting_elo, simulation_wins = forecast_game_elo(row['Home'], row['Visitor'], row['Season'], df, idx, starting_elo, simulation_wins)  

    return df, starting_elo, simulation_wins

In [139]:
elo = final_season_elos[2021]
df2 = scores_df[scores_df['Season'] == 2022]
num_simulations = 100
team_wins = {team: np.zeros(num_simulations) for team in df2['Home'].unique()}

for simulation in range(num_simulations):
    season_2022, elo_end, simulation_wins = season_forecaster(df2, elo)

    for team, wins in simulation_wins.items():
        team_wins[team][simulation] = wins


for team, wins in team_wins.items():
    mean_wins = np.mean(wins)
    std_deviation = np.std(wins)
    print(f"{team}: Mean Wins = {mean_wins}, Standard Deviation = {std_deviation}")

Tampa Bay Lightning: Mean Wins = 42.56, Standard Deviation = 7.647640158898691
Vegas Golden Knights: Mean Wins = 45.31, Standard Deviation = 9.723882969266958
Anaheim Ducks: Mean Wins = 45.44, Standard Deviation = 6.4935660464801614
Colorado Avalanche: Mean Wins = 36.14, Standard Deviation = 10.367275437645128
Edmonton Oilers: Mean Wins = 38.31, Standard Deviation = 8.490812681952182
Toronto Maple Leafs: Mean Wins = 40.96, Standard Deviation = 7.948484132210368
Washington Capitals: Mean Wins = 41.51, Standard Deviation = 9.428144037932386
Buffalo Sabres: Mean Wins = 46.58, Standard Deviation = 9.056688136399531
Carolina Hurricanes: Mean Wins = 40.66, Standard Deviation = 6.411271324784189
Columbus Blue Jackets: Mean Wins = 46.21, Standard Deviation = 8.645571120521767
Detroit Red Wings: Mean Wins = 39.81, Standard Deviation = 10.755180147259274
Florida Panthers: Mean Wins = 40.65, Standard Deviation = 9.60143218483576
Los Angeles Kings: Mean Wins = 39.28, Standard Deviation = 8.1487176

In [128]:
season_2022

,Date,Visitor,G_Vis,Home,G_Home,OT Ind,Season,EloDiffHome,EloDiffAway,WinProbHome,WinProbVis
23662,2021-10-12,Pittsburgh Penguins,3.0,Tampa Bay Lightning,2.0,1,2022,27.971830,-27.971830,0.540168,0.459832
23663,2021-10-12,Seattle Kraken,3.0,Vegas Golden Knights,4.0,1,2022,40.276999,-40.276999,0.557705,0.442295
23664,2021-10-13,Winnipeg Jets,2.0,Anaheim Ducks,1.0,1,2022,-55.252197,55.252197,0.421149,0.578851
23665,2021-10-13,Chicago Blackhawks,0.0,Colorado Avalanche,4.0,0,2022,246.573137,-246.573137,0.805243,0.194757
23666,2021-10-13,Vancouver Canucks,4.0,Edmonton Oilers,3.0,0,2022,73.805312,-73.805312,0.604645,0.395355
...,...,...,...,...,...,...,...,...,...,...,...
24969,2022-04-29,San Jose Sharks,1.0,Seattle Kraken,2.0,0,2022,166.276846,-166.276846,0.722554,0.277446
24970,2022-04-29,Vegas Golden Knights,2.0,St. Louis Blues,3.0,0,2022,-112.662536,112.662536,0.343319,0.656681
24971,2022-04-29,Boston Bruins,2.0,Toronto Maple Leafs,5.0,0,2022,118.108164,-118.108164,0.663713,0.336287
24972,2022-04-29,Calgary Flames,4.0,Winnipeg Jets,5.0,1,2022,63.103582,-63.103582,0.589828,0.410172
